In [1]:
# import relevant packages and modules
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import InvalidSelectorException, NoSuchElementException, TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import pygetwindow as gw
import time 

In [2]:
chrome_driverpath = "C:/chromedriver-win64/chromedriver.exe"
service = Service(executable_path=chrome_driverpath)
driver1 = webdriver.Chrome(service=service) 

website1= "https://www.transfermarkt.com/manchester-united/alumni/verein/985#google_vignette"
driver1.get(website1)
driver1.maximize_window()

In [3]:
# Wait setup
wait = WebDriverWait(driver1, 10)

try:
    # Wait for the iframe using @id 
    iframe = wait.until(EC.presence_of_element_located((By.XPATH, '//iframe[contains(@id, "sp_message")]')))
    driver1.switch_to.frame(iframe)
    print("✅ Switched to iframe.")

    # Wait for the Accept button and click it
    accept_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Accept All") or contains(text(), "Accept")]')))
    accept_btn.click()
    print("✅ Accept button clicked.")

    # Return to main content
    driver1.switch_to.default_content()
    print("✅ Switched back to main content.")

except TimeoutException:
    print("⚠️ Timed out waiting for iframe or accept button — maybe already accepted or not shown.")

# Continue scraping here...


✅ Switched to iframe.
✅ Accept button clicked.
✅ Switched back to main content.


In [4]:
detailed_button = wait.until(EC.element_to_be_clickable((By.XPATH, '(//div[@class="tm-tabs"]/a/div/span)[2]')))
detailed_button.click()
print("✅ On Detailed Page")

✅ On Detailed Page


In [21]:
# Wait briefly to allow pop-up iframe to load
time.sleep(3)

# Try to close the Google Ad popup if it appears
try:
    # Switch to the iframe containing the ad
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//iframe[contains(@id, "google_ads")]')))

    # Wait for the dismiss button to appear and click it
    dismiss_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="dismiss-button"]/div')))
    dismiss_btn.click()
    print("✅ Pop-up closed")

    # Switch back to the main content
    driver1.switch_to.default_content()

except Exception as e:
    print("⚠️ Pop-up not found or already dismissed:")

⚠️ Pop-up not found or already dismissed:


In [32]:
def close_google_ad_popup(driver, timeout=10):
    wait = WebDriverWait(driver, timeout)
    try:
        print("Attempting to close pop-up...")
        iframes = driver.find_elements(By.TAG_NAME, "iframe")
        print(f"Found {len(iframes)} iframes")
        
        for index, iframe in enumerate(iframes):
            try:
                driver.switch_to.frame(iframe)
                print(f"Switched to iframe {index}")
                
                dismiss_selectors = [
                    (By.XPATH, '//div[contains(@class, "close") or contains(@class, "dismiss")]'),
                    (By.XPATH, '//button | //div[@role="button"][contains(text(), "Close") or contains(text(), "Dismiss")]'),
                ]
                
                for by, selector in dismiss_selectors:
                    try:
                        dismiss_btn = wait.until(EC.element_to_be_clickable((by, selector)))
                        dismiss_btn.click()
                        print(f"✅ Pop-up closed with selector: {selector}")
                        return True
                    except TimeoutException:
                        print(f"Selector {selector} not found in iframe {index}")
                        continue
                
                driver.switch_to.default_content()
            except NoSuchFrameException:
                print(f"Iframe {index} is not accessible")
                driver.switch_to.default_content()
                continue
        
        print("⚠️ No dismiss button found in any iframe")
        return False
    
    except Exception as e:
        print(f"⚠️ Error closing pop-up: {str(e)}")
        return False
    finally:
        driver.switch_to.default_content()
        print("Returned to main content")

# Example usage
try:
    close_google_ad_popup(driver1, timeout=10)
except Exception as e:
    print(f"Failed to close pop-up: {str(e)}")

Attempting to close pop-up...
Found 10 iframes
Switched to iframe 0
Selector //div[contains(@class, "close") or contains(@class, "dismiss")] not found in iframe 0
Selector //button | //div[@role="button"][contains(text(), "Close") or contains(text(), "Dismiss")] not found in iframe 0
Switched to iframe 1
Selector //div[contains(@class, "close") or contains(@class, "dismiss")] not found in iframe 1
Selector //button | //div[@role="button"][contains(text(), "Close") or contains(text(), "Dismiss")] not found in iframe 1
Switched to iframe 2
Selector //div[contains(@class, "close") or contains(@class, "dismiss")] not found in iframe 2
Selector //button | //div[@role="button"][contains(text(), "Close") or contains(text(), "Dismiss")] not found in iframe 2
Switched to iframe 3
Selector //div[contains(@class, "close") or contains(@class, "dismiss")] not found in iframe 3
Selector //button | //div[@role="button"][contains(text(), "Close") or contains(text(), "Dismiss")] not found in iframe 3
S

In [30]:
len(iframes)

1

In [ ]:
# wait.until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//iframe[contains(@id, "google_ads")]')))
dismiss_btn = driver1.find_element(By.XPATH, '//*[@id="dismiss-button"]/div')
dismiss_btn.click()
# print("✅ Pop-up closed")

In [6]:
# dismiss_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="dismiss-button"]/div')))

In [7]:
# just to be sure
driver1.switch_to.default_content()

In [8]:
# # accept cookies
# accept_cookies_button = driver1.find_element(By.PARTIAL_LINK_TEXT,"Accept&continue")
# accept_cookies_button.click()

In [9]:
# # Accept cookies (fixed selector)
# accept_cookies_button = driver1.find_element(By.XPATH, '//button[@title="Accept & continue"][1]')
# accept_cookies_button.click()


In [10]:
# iframe = driver1.find_element(By.XPATH, '//iframe[contains(@id, "sp_message")]')
# driver1.switch_to.frame(iframe)
# print("✅ Switched to iframe.")

In [11]:
# accept_btn = driver1.find_element(By.XPATH, '//button[contains(text(), "Accept")]')
# accept_btn.click()
# print("✅ Accept button clicked.")

In [33]:
all_names, all_positions, all_countries, all_dobs, all_teams = [], [], [], [], []
for page in range(1,3):
    print(f"On page{page}")
    try:
        # player names
        player_names = driver1.find_elements(By.XPATH,'//tbody/tr/td[1]/table[1][@class="inline-table"]/tbody/tr/td[2]')
        # position
        player_positions = driver1.find_elements(By.XPATH, '//tbody/tr/td[1]/table[1][@class="inline-table"]/tbody/tr[2]/td')
        # country
        player_countries = driver1.find_elements(By.XPATH,'//table[@class="items"]/tbody/tr/td[5]/img')
        # age
        player_dobs = driver1.find_elements(By.XPATH,'//*[@id="yw1"]/table/tbody/tr/td[3]')
        # Teams
        player_teams = driver1.find_elements(By.XPATH,'//table[@class="items"]/tbody/tr/td[8]')

        # Append data
        all_names.extend([n.text for n in player_names])
        all_positions.extend([p.text for p in player_positions])
        all_countries.extend([c.get_attribute('title') for c in player_countries])
        all_dobs.extend([d.text for d in player_dobs])
        all_teams.extend([t.text for t in player_teams])

        print(f"✅ Page {page} scraped: {len(player_names)} players.")
    except Exception as e:
        print(f"❌ Error extracting data on page {page}: {e}")    
    try:
        wait = WebDriverWait(driver1, 10)
        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@title="Go to the next page"]')))
        driver1.execute_script("arguments[0].click();", next_button)
    except:
        print("Next button error")
    time.sleep(20)

        

On page1
✅ Page 1 scraped: 25 players.
On page2
✅ Page 2 scraped: 25 players.


In [34]:
print(len(all_names))
print(len(all_positions))
print(len(all_countries))
print(len(all_dobs))
print(len(all_teams))

50
50
50
50
50


In [35]:
def has_duplicates(lst):
    return len(lst) != len(set(lst))
print(has_duplicates(all_names)) # Output: True

False


In [ ]:
all_names, all_positions, all_countries, all_dobs, all_teams = [], [], [], [], []

wait = WebDriverWait(driver1, 10)

for page in range(1, 4):
    print(f"📄 On page {page}")
    try:
        # Wait for table to load by checking for a stable element
        wait.until(EC.presence_of_element_located((By.XPATH, '//table[@class="items"]/tbody/tr')))

        # Scrape data
        player_names = driver1.find_elements(By.XPATH,'//tbody/tr/td[1]/table[1][@class="inline-table"]/tbody/tr/td[2]')
        player_positions = driver1.find_elements(By.XPATH,'//tbody/tr/td[1]/table[1][@class="inline-table"]/tbody/tr[2]/td')
        player_countries = driver1.find_elements(By.XPATH,'//table[@class="items"]/tbody/tr/td[5]/img')
        player_dobs = driver1.find_elements(By.XPATH,'//*[@id="yw1"]/table/tbody/tr/td[3]')
        player_teams = driver1.find_elements(By.XPATH,'//table[@class="items"]/tbody/tr/td[8]')

        # Append data
        all_names.extend([n.text for n in player_names])
        all_positions.extend([p.text for p in player_positions])
        all_countries.extend([c.get_attribute('title') for c in player_countries])
        all_dobs.extend([d.text for d in player_dobs])
        all_teams.extend([t.text for t in player_teams])

        print(f"✅ Scraped Page {page}: {len(player_names)} players")

    except Exception as e:
        print(f"❌ Error scraping data on page {page}: {e}")

    # Try to go to the next page unless it's the last one
    if page < 2:
        try:
            # Hide potential ad iframe that might block the next button
            driver1.execute_script("""
                var iframe = document.querySelector('iframe[id^="google_ads_iframe"]');
                if (iframe) {
                    iframe.style.display = 'none';
                }
            """)
            # Wait and click next button
            next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@title="Go to the next page"]')))
            driver1.execute_script("arguments[0].click();", next_button)
            print("➡️ Clicked next page button")

        except Exception as e:
            print(f"⚠️ Couldn't click next: {e}")
    time.sleep(20)


In [15]:
all_names
# all_positions
# all_countries
# all_dobs
# all_teams

['Cristiano Ronaldo',
 'Raphaël Varane',
 'Paul Pogba',
 'Jadon Sancho',
 'Gerard Piqué',
 'Casemiro',
 'David Beckham',
 'Ángel Di María',
 'Juan Mata',
 'Christian Eriksen',
 'Romelu Lukaku',
 'Marcus Rashford',
 'Lisandro Martínez',
 'Bruno Fernandes',
 'Mason Mount',
 'Wayne Rooney',
 'David de Gea',
 'Bastian Schweinsteiger',
 'Víctor Valdés',
 'Radamel Falcao',
 'Anthony Martial',
 'Matthijs de Ligt',
 'Antony',
 'Zlatan Ibrahimović',
 'Alexis Sánchez',
 'Harry Maguire',
 'Carlos Tevez',
 'Memphis Depay',
 'Ryan Giggs',
 'Joshua Zirkzee',
 'Rasmus Højlund',
 'Edinson Cavani',
 'Nani',
 'Paul Scholes',
 'Manuel Ugarte',
 'Robin van Persie',
 'Alex Telles',
 'Owen Hargreaves',
 'Nemanja Vidic',
 'Wilfried Zaha',
 'Donny van de Beek',
 'Kobbie Mainoo',
 'Leny Yoro',
 'Henrikh Mkhitaryan',
 'Luke Shaw',
 'Rio Ferdinand',
 'Fabien Barthez',
 'Jesse Lingard',
 'Fred',
 'Michael Carrick',
 'Kléberson',
 'Alejandro Garnacho',
 'Mason Greenwood',
 'Anderson',
 'Ron-Robert Zieler',
 'Roy K

In [55]:
# player names
player_names = driver1.find_elements(By.XPATH,'//tbody/tr/td[1]/table[1][@class="inline-table"]/tbody/tr/td[2]')
# position
player_position = driver1.find_elements(By.XPATH, '//tbody/tr/td[1]/table[1][@class="inline-table"]/tbody/tr[2]/td')
# country
player_countries = driver1.find_elements(By.XPATH,'//table[@class="items"]/tbody/tr/td[5]/img')
# age
player_dob = driver1.find_elements(By.XPATH,'//*[@id="yw1"]/table/tbody/tr/td[3]')
# Teams
player_teams = driver1.find_elements(By.XPATH,'//table[@class="items"]/tbody/tr/td[8]')

25
25
25
25
25


In [55]:
# player names
player_names = driver1.find_elements(By.XPATH,'//tbody/tr/td[1]/table[1][@class="inline-table"]/tbody/tr/td[2]')
print(len(player_names))

# position
player_position = driver1.find_elements(By.XPATH, '//tbody/tr/td[1]/table[1][@class="inline-table"]/tbody/tr[2]/td')
print(len(player_position))

# country
player_countries = driver1.find_elements(By.XPATH,'//table[@class="items"]/tbody/tr/td[5]/img')
print(len(player_countries))

# age
player_dob = driver1.find_elements(By.XPATH,'//*[@id="yw1"]/table/tbody/tr/td[3]')
print(len(player_dob))

# Teams
player_teams = driver1.find_elements(By.XPATH,'//table[@class="items"]/tbody/tr/td[8]')
print(len(player_teams))

25
25
25
25
25


In [56]:
for i,name in zip(range(1,26),player_dob):
    print(f"{i}.{name.text}")

1.Feb 5, 1985
2.Apr 25, 1993
3.Mar 15, 1993
4.Mar 25, 2000
5.Feb 2, 1987
6.Feb 23, 1992
7.May 2, 1975
8.Feb 14, 1988
9.Apr 28, 1988
10.Feb 14, 1992
11.May 13, 1993
12.Oct 31, 1997
13.Jan 18, 1998
14.Sep 8, 1994
15.Jan 10, 1999
16.Oct 24, 1985
17.Nov 7, 1990
18.Aug 1, 1984
19.Jan 14, 1982
20.Feb 10, 1986
21.Dec 5, 1995
22.Aug 12, 1999
23.Feb 24, 2000
24.Oct 3, 1981
25.Dec 19, 1988


In [59]:

wait = WebDriverWait(driver1, 10)
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@title="Go to the next page"]')))
driver1.execute_script("arguments[0].click();", next_button)


In [33]:
next_page1 = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="yw1"]/div[2]/ul/li[11]/a')))
next_page1.click()

In [36]:
next_page2 = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="yw1"]/div[2]/ul/li[13]/a')))
next_page2.click()

In [50]:
player_ages = driver1.find_elements(By.XPATH,'//*[@id="yw1"]/table/tbody/tr/td[3]')
player_ages[0].text

'40'

In [16]:
# Create DataFrame and Save
df = pd.DataFrame({
    "Name": all_names,
    "Position": all_positions,
    "Country": all_countries,
    "DOB/Age": all_dobs,
    "Team": all_teams
})

In [20]:
df.head()

,Name,Position,Country,DOB/Age,Team
0,Cristiano Ronaldo,Centre-Forward,Portugal,"Feb 5, 1985",First Team
1,Raphaël Varane,Centre-Back,France,"Apr 25, 1993",First Team
2,Paul Pogba,Central Midfield,France,"Mar 15, 1993",First Team/Reserves Team/Youth Teams
3,Jadon Sancho,Left Winger,England,"Mar 25, 2000",First Team
4,Gerard Piqué,Centre-Back,Spain,"Feb 2, 1987",First Team
